# Machine Learning Engineer Nanodegree
## Model Evaluation & Validation
## Project: Predicting Boston Housing Prices

Welcome to the first project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and you will need to implement additional functionality to successfully complete this project. You will not need to modify the included code beyond what is requested. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a 'TODO' statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

## Getting Started
In this project, you will evaluate the performance and predictive power of a model that has been trained and tested on data collected from homes in suburbs of Boston, Massachusetts. A model trained on this data that is seen as a *good fit* could then be used to make certain predictions about a home — in particular, its monetary value. This model would prove to be invaluable for someone like a real estate agent who could make use of such information on a daily basis.

The dataset for this project originates from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/Housing). The Boston housing data was collected in 1978 and each of the 506 entries represent aggregated data about 14 features for homes from various suburbs in Boston, Massachusetts. For the purposes of this project, the following preprocessing steps have been made to the dataset:
- 16 data points have an `'MEDV'` value of 50.0. These data points likely contain **missing or censored values** and have been removed.
- 1 data point has an `'RM'` value of 8.78. This data point can be considered an **outlier** and has been removed.
- The features `'RM'`, `'LSTAT'`, `'PTRATIO'`, and `'MEDV'` are essential. The remaining **non-relevant features** have been excluded.
- The feature `'MEDV'` has been **multiplicatively scaled** to account for 35 years of market inflation.

Run the code cell below to load the Boston housing dataset, along with a few of the necessary Python libraries required for this project. You will know the dataset loaded successfully if the size of the dataset is reported.

In [ ]:
# Import libraries necessary for this project
import numpy as np
import pandas as pd
from sklearn.cross_validation import ShuffleSplit
import matplotlib.pyplot as plt

# Import supplementary visualizations code visuals.py
import visuals as vs

# Pretty display for notebooks
%matplotlib inline

# Load the Boston housing dataset
data = pd.read_csv('../input/housing.csv')
prices = data['MEDV']
features = data.drop('MEDV', axis = 1)
    
# Success
print ("Boston housing dataset has {} data points with {} variables each.".format(*data.shape))


## Data Exploration
In this first section of this project, you will make a cursory investigation about the Boston housing data and provide your observations. Familiarizing yourself with the data through an explorative process is a fundamental practice to help you better understand and justify your results.

Since the main goal of this project is to construct a working model which has the capability of predicting the value of houses, we will need to separate the dataset into **features** and the **target variable**. The **features**, `'RM'`, `'LSTAT'`, and `'PTRATIO'`, give us quantitative information about each data point. The **target variable**, `'MEDV'`, will be the variable we seek to predict. These are stored in `features` and `prices`, respectively.

### Implementation: Calculate Statistics
For your very first coding implementation, you will calculate descriptive statistics about the Boston housing prices. Since `numpy` has already been imported for you, use this library to perform the necessary calculations. These statistics will be extremely important later on to analyze various prediction results from the constructed model.

In the code cell below, you will need to implement the following:
- Calculate the minimum, maximum, mean, median, and standard deviation of `'MEDV'`, which is stored in `prices`.
  - Store each calculation in their respective variable.

In [ ]:
# TODO: Minimum price of the data
minimum_price = np.amin(prices)

# TODO: Maximum price of the data
maximum_price = np.amax(prices)

# TODO: Mean price of the data
mean_price = np.mean(prices)

# TODO: Median price of the data
median_price = np.median(prices)

# TODO: Standard deviation of prices of the data
std_price = np.std(prices)

# Show the calculated statistics
print ("Statistics for Boston housing dataset:\n")
print ("Minimum price: ${:,.2f}".format(minimum_price))
print ("Maximum price: ${:,.2f}".format(maximum_price))
print ("Mean price: ${:,.2f}".format(mean_price))
print ("Median price ${:,.2f}".format(median_price))
print ("Standard deviation of prices: ${:,.2f}".format(std_price))

### Question 1 - Feature Observation
As a reminder, we are using three features from the Boston housing dataset: `'RM'`, `'LSTAT'`, and `'PTRATIO'`. For each data point (neighborhood):
- `'RM'` is the average number of rooms among homes in the neighborhood.
- `'LSTAT'` is the percentage of homeowners in the neighborhood considered "lower class" (working poor).
- `'PTRATIO'` is the ratio of students to teachers in primary and secondary schools in the neighborhood.

_Using your intuition, for each of the three features above, do you think that an increase in the value of that feature would lead to an **increase** in the value of `'MEDV'` or a **decrease** in the value of `'MEDV'`? Justify your answer for each._  
**Hint:** Would you expect a home that has an `'RM'` value of 6 be worth more or less than a home that has an `'RM'` value of 7?

**Answer: **

The value of 'MEDV' is the price of the home. Prices of homes are primarily affected by location, size, municipal services, and class of neighborhood. 

'RM' is a rough indicator of the house size, i.e. the larger the amount of rooms that it has the larger it would be. Therefore, an increase in this value is expected to increase the value of 'MEDV'.

'LSTAT' is an indicator of the class of the respective neighborhood. Meaning if the percentage of homeowners that are considered "lower class" is higher the class of the neighborhood is lower. Therefore, an increase of 'LSTAT' will decrease the value of 'MEDV'.

'PTRATIO' is an indicator of how satisfactory the education system of the neighborhood is. The fewer students you have per teacher the more attention students recieve, and it indicates of high funding to the education system. Therefore, a decrease in this value will lead to an increase in the value of 'MEDV'.

----

## Developing a Model
In this second section of the project, you will develop the tools and techniques necessary for a model to make a prediction. Being able to make accurate evaluations of each model's performance through the use of these tools and techniques helps to greatly reinforce the confidence in your predictions.

### Implementation: Define a Performance Metric
It is difficult to measure the quality of a given model without quantifying its performance over training and testing. This is typically done using some type of performance metric, whether it is through calculating some type of error, the goodness of fit, or some other useful measurement. For this project, you will be calculating the [*coefficient of determination*](http://stattrek.com/statistics/dictionary.aspx?definition=coefficient_of_determination), R<sup>2</sup>, to quantify your model's performance. The coefficient of determination for a model is a useful statistic in regression analysis, as it often describes how "good" that model is at making predictions. 

The values for R<sup>2</sup> range from 0 to 1, which captures the percentage of squared correlation between the predicted and actual values of the **target variable**. A model with an R<sup>2</sup> of 0 is no better than a model that always predicts the *mean* of the target variable, whereas a model with an R<sup>2</sup> of 1 perfectly predicts the target variable. Any value between 0 and 1 indicates what percentage of the target variable, using this model, can be explained by the **features**. _A model can be given a negative R<sup>2</sup> as well, which indicates that the model is **arbitrarily worse** than one that always predicts the mean of the target variable._

For the `performance_metric` function in the code cell below, you will need to implement the following:
- Use `r2_score` from `sklearn.metrics` to perform a performance calculation between `y_true` and `y_predict`.
- Assign the performance score to the `score` variable.

In [ ]:
# TODO: Import 'r2_score'
from sklearn.metrics import r2_score

def performance_metric(y_true, y_predict):
    """ Calculates and returns the performance score between 
        true and predicted values based on the metric chosen. """
    
    # TODO: Calculate the performance score between 'y_true' and 'y_predict'
    score = r2_score(y_true, y_predict)
    
    # Return the score
    return score

### Question 2 - Goodness of Fit
Assume that a dataset contains five data points and a model made the following predictions for the target variable:

| True Value | Prediction |
| :-------------: | :--------: |
| 3.0 | 2.5 |
| -0.5 | 0.0 |
| 2.0 | 2.1 |
| 7.0 | 7.8 |
| 4.2 | 5.3 |
*Would you consider this model to have successfully captured the variation of the target variable? Why or why not?* 

Run the code cell below to use the `performance_metric` function and calculate this model's coefficient of determination.

In [ ]:
# Calculate the performance of this model
score = performance_metric([3, -0.5, 2, 7, 4.2], [2.5, 0.0, 2.1, 7.8, 5.3])
print ("Model has a coefficient of determination, R^2, of {:.3f}.".format(score))

**Answer:**

I would consider the model to have correctly captured the variation of the target variable, since the coefficient of determination is very high, 0.923. Even just by glancing at the numbers it can be infered that the model is quite accurate. If we show it graphically it would be more appearent. 

Another insight is that more data points would have allowed us to capture more of the variability of the data, since there would be more data points to compare to. This would give us a more precise estimate of the R2 score, which would probably be lower since the model won't overfit as much as it did with a smaller amount of data points. 

In [ ]:
a=[3, -0.5, 2, 7, 4.2]
b=[2.5, 0.0, 2.1, 7.8, 5.3]
plt.figure()
plt.plot(a, '-o', b, '-o')
plt.gca().fill_between(range(len(a)), 
                       a, b, 
                       facecolor='blue', 
                       alpha=0.25)
plt.title('True Value vs. Prediction')
plt.xlabel('Data')
plt.ylabel('Values')
plt.legend(['True Value', 'Prediction'])

**About the Graph**
As we can see, the points are very correlated to each other in the same data point. That is each point correlates with the other one very precisely. This means our prediction is a good model of the underlying data. The color filled between magnifies the difference, which is very small.

### Implementation: Shuffle and Split Data
Your next implementation requires that you take the Boston housing dataset and split the data into training and testing subsets. Typically, the data is also shuffled into a random order when creating the training and testing subsets to remove any bias in the ordering of the dataset.

For the code cell below, you will need to implement the following:
- Use `train_test_split` from `sklearn.cross_validation` to shuffle and split the `features` and `prices` data into training and testing sets.
  - Split the data into 80% training and 20% testing.
  - Set the `random_state` for `train_test_split` to a value of your choice. This ensures results are consistent.
- Assign the train and testing splits to `X_train`, `X_test`, `y_train`, and `y_test`.

In [ ]:
# TODO: Import 'train_test_split'
from sklearn.cross_validation import train_test_split 
# TODO: Shuffle and split the data into training and testing subsets
X_train, X_test, y_train, y_test = train_test_split(features, prices, test_size=0.2, random_state=40)

# Success
print ("Training and testing split was successful.")

### Question 3 - Training and Testing
*What is the benefit to splitting a dataset into some ratio of training and testing subsets for a learning algorithm?*  
**Hint:** What could go wrong with not having a way to test your model?

**Answer: **

The two causes of error are bias and variation. If we didn't have a testing dataset, the variance would be too high, indicating overfitting. We wouldn't have had an accurate method to check if we are actually predicting correct results, because we would've only had the data we trained on to validate our model.

The ratio is vital since this is what we allocate the model to train on. If it's too large then the model will underfit, and overfit vice versa.

This is why we need a testing dataset where the model was not trained on (using the .fit() method) to check whether the parameters we used for the model are valid and there is no overfitting. We could just have a wrong model for prediction. In which case wee would have to got through model selection, described in question 8. 

----

## Analyzing Model Performance
In this third section of the project, you'll take a look at several models' learning and testing performances on various subsets of training data. Additionally, you'll investigate one particular algorithm with an increasing `'max_depth'` parameter on the full training set to observe how model complexity affects performance. Graphing your model's performance based on varying criteria can be beneficial in the analysis process, such as visualizing behavior that may not have been apparent from the results alone.

### Learning Curves
The following code cell produces four graphs for a decision tree model with different maximum depths. Each graph visualizes the learning curves of the model for both training and testing as the size of the training set is increased. Note that the shaded region of a learning curve denotes the uncertainty of that curve (measured as the standard deviation). The model is scored on both the training and testing sets using R<sup>2</sup>, the coefficient of determination.  

Run the code cell below and use these graphs to answer the following question.

In [ ]:
# Produce learning curves for varying training set sizes and maximum depths
vs.ModelLearning(features, prices)

### Question 4 - Learning the Data
*Choose one of the graphs above and state the maximum depth for the model. What happens to the score of the training curve as more training points are added? What about the testing curve? Would having more training points benefit the model?*  
**Hint:** Are the learning curves converging to particular scores?

**Answer: **

In the graph that describes max_depth=1, We can see that as we add more training points two trends are taking place. Firstly, the testing score increases until it converges to around 0.4, with some deviation. Secondly, the training score decreases from a perfect one to around 0.45, right above the testing score. 

In infinitesimal mathematics, this can be described as the testing score function converging to 0.4 as the number of training points approaches infinity. Same applies to the testing score function, which approaches 0.45 as the number of training points approaches infinity. 

This implies that there is a limit of training points where there would be real value added to the model. In this graph that threshold be around 250  training points. Adding more data points is expensive copmutationally, and time-consuming to collect all of the data. Checking for convergence prevents us from having to fall into these inefficient procedures.

### Complexity Curves
The following code cell produces a graph for a decision tree model that has been trained and validated on the training data using different maximum depths. The graph produces two complexity curves — one for training and one for validation. Similar to the **learning curves**, the shaded regions of both the complexity curves denote the uncertainty in those curves, and the model is scored on both the training and validation sets using the `performance_metric` function.  

Run the code cell below and use this graph to answer the following two questions.

In [ ]:
vs.ModelComplexity(X_train, y_train)

### Question 5 - Bias-Variance Tradeoff
*When the model is trained with a maximum depth of 1, does the model suffer from high bias or from high variance? How about when the model is trained with a maximum depth of 10? What visual cues in the graph justify your conclusions?*  
**Hint:** How do you know when a model is suffering from high bias or high variance?

**Answer: **


To recap, high bias means the model is underfitting the data, which means that it's not correlated sufficiently. On the contrary, high variance means that it's overfitting, which means it overlooks at small differences in the data and is oversensitive.

For instance, when the maximum depth is 1, there is high bias. The model is not correlated to the actual data precisely. This can be seen visually that the training score is distanced away from one. This means that the model didn't fit itself to the training data accurately.

On the other hand, when the maximum depth is 10, there is high variance. The model is too sensitive to the training data. The decision tree model uses too many intricate rules that are too specific to the data and don't accurately capture the features. Visually, this can be inferred from the model having a perfect one training score although it's distant from the testing score, which is at around 0.65. 

Overall, the optimal balance is when the training score is close to perfect, i.e. 1, and is proximate to the testing score. Theoretically, this indicates a goot fit of the training data and correct depiction of the underlying features, as the testing data also has a high score.

### Question 6 - Best-Guess Optimal Model
*Which maximum depth do you think results in a model that best generalizes to unseen data? What intuition lead you to this answer?*

**Answer: **

Based on the visualization, the variance increases between max depth 3 and max depth 6, in conjunction with the bias decreasing. Therefore, in that range there would be the optimal depth. In summary, the optimal maximum depth would either be 4 or 5.

-----

## Evaluating Model Performance
In this final section of the project, you will construct a model and make a prediction on the client's feature set using an optimized model from `fit_model`.

### Question 7 - Grid Search
*What is the grid search technique and how it can be applied to optimize a learning algorithm?*

**Answer: **

The grid search technique exhaustively checks the the learning algorithm with the grid of parameter values we specified in the param_grid parameter.  

The grid search retains the best combination of parameter values, based on the best scoring function result, and returns grid search object with this combination.

### Question 8 - Cross-Validation
*What is the k-fold cross-validation training technique? What benefit does this technique provide for grid search when optimizing a model?*  
**Hint:** Much like the reasoning behind having a testing set, what could go wrong with using grid search without a cross-validated set?

**Answer: **

The k-fold cross validation technique is another way of reducing error when training our data. In addition to using a training and testing datasets for validation, we are splitting the complete dataset k times into training and testing. This means each data point will be used for training k-1 times and for testing once. As k increases, the variance in the result decreases, retaining a more accurate validation. Intuitivially, this means we're not relying on any particular subset of the data for testing, we are checking our model on all parts of the data, in comparison to the remaining data used for training. 

We need to choose the number of splits correctly, because too many splits means we are computationally expensive, as it means we are performing multiple operations from scratch. Yet, if we split into too little chunks, i.e. many splits, the cross validation wouldn't be as accurate since our testing set would be too small compared to the training set.

During the last step, we are taking the avearge of the k results to produce the estimator. We get an error rate for each fold, or part, of the dataset and, for each model, take that average and compare to the average of other models. Finally, we can compare this error rate to untouched data to check its accuracy, as the average error rate from cross validation still has some bias.

In this project we are using a single model, i.e. Decision Tree Algorithm, to train our data. We are optimizing it using grid search, as mentioned in question 7, to pick the best 'max_depth' based on the error rates for each depth. We could have used an additional model, such as polynomial regression, and check its average error rate compares to the decision tree algorithm, and then we would have decided between the two, which is the optimal one for use in this dataset.


### Implementation: Fitting a Model
Your final implementation requires that you bring everything together and train a model using the **decision tree algorithm**. To ensure that you are producing an optimized model, you will train the model using the grid search technique to optimize the `'max_depth'` parameter for the decision tree. The `'max_depth'` parameter can be thought of as how many questions the decision tree algorithm is allowed to ask about the data before making a prediction. Decision trees are part of a class of algorithms called *supervised learning algorithms*.

In addition, you will find your implementation is using `ShuffleSplit()` for an alternative form of cross-validation (see the `'cv_sets'` variable). While it is not the K-Fold cross-validation technique you describe in **Question 8**, this type of cross-validation technique is just as useful!. The `ShuffleSplit()` implementation below will create 10 (`'n_splits'`) shuffled sets, and for each shuffle, 20% (`'test_size'`) of the data will be used as the *validation set*. While you're working on your implementation, think about the contrasts and similarities it has to the K-fold cross-validation technique.

Please note that ShuffleSplit has different parameters in scikit-learn versions 0.17 and 0.18.
For the `fit_model` function in the code cell below, you will need to implement the following:
- Use [`DecisionTreeRegressor`](http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html) from `sklearn.tree` to create a decision tree regressor object.
  - Assign this object to the `'regressor'` variable.
- Create a dictionary for `'max_depth'` with the values from 1 to 10, and assign this to the `'params'` variable.
- Use [`make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html) from `sklearn.metrics` to create a scoring function object.
  - Pass the `performance_metric` function as a parameter to the object.
  - Assign this scoring function to the `'scoring_fnc'` variable.
- Use [`GridSearchCV`](http://scikit-learn.org/0.17/modules/generated/sklearn.grid_search.GridSearchCV.html) from `sklearn.grid_search` to create a grid search object.
  - Pass the variables `'regressor'`, `'params'`, `'scoring_fnc'`, and `'cv_sets'` as parameters to the object. 
  - Assign the `GridSearchCV` object to the `'grid'` variable.

In [ ]:
# TODO: Import 'make_scorer', 'DecisionTreeRegressor', and 'GridSearchCV'
from sklearn.metrics import make_scorer
from sklearn.grid_search import GridSearchCV
from sklearn.tree import DecisionTreeRegressor
def fit_model(X, y):
    """ Performs grid search over the 'max_depth' parameter for a 
        decision tree regressor trained on the input data [X, y]. """
    
    # Create cross-validation sets from the training data
    cv_sets = ShuffleSplit(X.shape[0], test_size = 0.20, random_state = 0)

    # TODO: Create a decision tree regressor object
    regressor = DecisionTreeRegressor()

    # TODO: Create a dictionary for the parameter 'max_depth' with a range from 1 to 10
    params = {  'max_depth': [i+1 for i in range(10)]  }

    # TODO: Transform 'performance_metric' into a scoring function using 'make_scorer' 
    scoring_fnc = make_scorer(performance_metric)

    # TODO: Create the grid search object
    grid = GridSearchCV(regressor, param_grid=params, cv=cv_sets, scoring=scoring_fnc)

    # Fit the grid search object to the data to compute the optimal model
    grid = grid.fit(X, y)

    # Return the optimal model after fitting the data
    return grid.best_estimator_

### Making Predictions
Once a model has been trained on a given set of data, it can now be used to make predictions on new sets of input data. In the case of a *decision tree regressor*, the model has learned *what the best questions to ask about the input data are*, and can respond with a prediction for the **target variable**. You can use these predictions to gain information about data where the value of the target variable is unknown — such as data the model was not trained on.

### Question 9 - Optimal Model
_What maximum depth does the optimal model have? How does this result compare to your guess in **Question 6**?_  

Run the code block below to fit the decision tree regressor to the training data and produce an optimal model.

In [ ]:
# Fit the training data to the model using grid search
reg = fit_model(X_train, y_train)

# Produce the value for 'max_depth'
print ("Parameter 'max_depth' is {} for the optimal model.".format(reg.get_params()['max_depth']))

**Answer: **

The optimal model has max depth of 4. This is close to my estimation in question 6 which was either 4 or 5. That estimation was beneficial as it gives me a measure of testing that I am using the correct methods in my implementation.

This means that this is the parameter in which the decision tree algorithm yields low bias and low variance.

### Question 10 - Predicting Selling Prices
Imagine that you were a real estate agent in the Boston area looking to use this model to help price homes owned by your clients that they wish to sell. You have collected the following information from three of your clients:

| Feature | Client 1 | Client 2 | Client 3 |
| :---: | :---: | :---: | :---: |
| Total number of rooms in home | 5 rooms | 4 rooms | 8 rooms |
| Neighborhood poverty level (as %) | 17% | 32% | 3% |
| Student-teacher ratio of nearby schools | 15-to-1 | 22-to-1 | 12-to-1 |
*What price would you recommend each client sell his/her home at? Do these prices seem reasonable given the values for the respective features?*  
**Hint:** Use the statistics you calculated in the **Data Exploration** section to help justify your response.  

Run the code block below to have your optimized model make predictions for each client's home.

In [ ]:
# Produce a matrix for client data
client_data = [[5, 17, 15], # Client 1
               [4, 32, 22], # Client 2
               [8, 3, 12]]  # Client 3

# Show predictions
for i, price in enumerate(reg.predict(client_data)):
    print ("Predicted selling price for Client {}'s home: ${:,.2f}".format(i+1, price))

**Answer: **

Disregarding the results obtained by the model, here are my suggestions for each client based on my observation in Data Exploration:
    1. There are 5 rooms which is sizeable, the poverty level is quie low, and the s-t ratio is average. The price should be around the median price.
    2. 4 rooms isn't many, the poverty level is very high (almost a third of residents in the neighborhood are in poverty), and the s-t ratio is terrible. The price should be very low, although not minimal.
    3. 8 rooms indicates it's a large house, the poverty level is extremely low (there are almost no residents in poverty in the respective neighborhood), and the s-t ratio is low. I expect the price to be very high.

The model seems to accurately predict the selling price I derived from my estimation of the affect of particular features on selling prices. The model has done this only with numerical calculation with no knowledge of the meaning of the statistics, which is very impressive.





### Sensitivity
An optimal model is not necessarily a robust model. Sometimes, a model is either too complex or too simple to sufficiently generalize to new data. Sometimes, a model could use a learning algorithm that is not appropriate for the structure of the data given. Other times, the data itself could be too noisy or contain too few samples to allow a model to adequately capture the target variable — i.e., the model is underfitted. Run the code cell below to run the `fit_model` function ten times with different training and testing sets to see how the prediction for a specific client changes with the data it's trained on.

In [ ]:
vs.PredictTrials(features, prices, fit_model, client_data)

### Question 11 - Applicability
*In a few sentences, discuss whether the constructed model should or should not be used in a real-world setting.*  
**Hint:** Some questions to answering:
- *How relevant today is data that was collected from 1978?*
- *Are the features present in the data sufficient to describe a home?*
- *Is the model robust enough to make consistent predictions?*
- *Would data collected in an urban city like Boston be applicable in a rural city?*

**Answer: **

Let's first summarize the methods we have used to derive our model and then discuss how they should be adjusted for real-life housing prediction. 

We are relying on a relatively small dataset, i.e. 489 data points, for a large city of millions of inhabitants. We are capturing 4 variables, of which 3 features are used to determine the price. We are using a decision tree with an r2 scorer to derive our model. We also found a moderate sensitivity in our model of about 70,000$.

In real-life, real estate prices are affected by macro fluctuations in the economy, the political state, and specifically there are 14 features in the original dataset that we havn't used. Housing prices differ drastically over time, so data from 35 years ago is absolutely irrelevant. There are hundreds of thousands of listings in urban areas like Boston, implying we need much more data. 

In order to better fit our model to a real-world setting we need:
    1. A larger dataset on the magnitude of hundreds of thousands of data points, that is up to date. Along with a secondary dataset of economic indicators.
    2. Decision trees are known to be NP-complete, i.e. they are based on heuristic algorithms that make local decisions at 
    each node. Consequently, they don't usually return a globally optimized decision tree. We should use principal component analysis for dimensionality reduction to avoid overfitting. 
    3. We need to use more features that describe real-estate commodities. These include crime rate, distance to employment centers, air pollution, and accessibility to highways.
    4. Take into account if the area is rural or urban and use specific features for each category.

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.